In [2]:
import tomopy
import dxchange
import matplotlib.pyplot as plt
import time
import binascii
import tempfile

/Users/giorgoschantzialexiou/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Spark lightsource-(Sirt,Gridrec) Total time to Completion 

In [3]:
def lightsource_reconstruction_spark(message, algo='gridrec'):
    
    msg_bin = binascii.unhexlify(message)
    tf = tempfile.NamedTemporaryFile(delete=True)
    tf.write(msg_bin)
    tf.flush()
    
    start = 0
    end = 2
    
    proj, flat, dark = dxchange.read_aps_32id(tf.name, sino=(start,end))
    theta = tomopy.angles(proj.shape[0])



    if (theta is None):
        theta = tomopy.angles(proj.shape[0])
    else:
        pass
    
    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm=algo)
    
    tf.close()
    
    return 0


## create message
data_file = 'tooth.h5'
with open(data_file,'r') as f:
    data = f.read()

data_enc = binascii.hexlify(data).encode('utf-8')

data_list = []
for i in range(1):
    data_list.append(data_enc)
    
data_list_rdd = sc.parallelize(data_list,len(data_list))

avg_gridrec  = 0
for i in range(10):
    tts = time.time()
    res = data_list_rdd.map(lightsource_reconstruction_spark).collect()
    tte = time.time()
    avg_gridrec += (tte - tts)
avg_gridrec /= 10
print 'Avg time for gridrec algorithm is:', str(avg_gridrec), ' in seconds'

    

Avg time for gridrec algorithm is: 2.61919734478  in seconds


In [4]:
def lightsource_reconstruction_spark(message, algo='sirt'):
    
    msg_bin = binascii.unhexlify(message)
    tf = tempfile.NamedTemporaryFile(delete=True)
    tf.write(msg_bin)
    tf.flush()
    
    start = 0
    end = 2
    
    proj, flat, dark = dxchange.read_aps_32id(tf.name, sino=(start,end))
    theta = tomopy.angles(proj.shape[0])



    if (theta is None):
        theta = tomopy.angles(proj.shape[0])
    else:
        pass
    
    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm=algo)
    
    tf.close()
    
    return 0


## create message
data_file = 'tooth.h5'
with open(data_file,'r') as f:
    data = f.read()

data_enc = binascii.hexlify(data).encode('utf-8')

data_list = []
for i in range(1):
    data_list.append(data_enc)

    
data_list_rdd = sc.parallelize(data_list,len(data_list))
avg_sirt  = 0
for i in range(10):
    tts = time.time()
    res = data_list_rdd.map(lightsource_reconstruction_spark).collect()
    tte = time.time()
    #print res
    #print 'sirt TTC is: ', (tte-tts)
    avg_sirt += (tte -tts)
avg_sirt /= 10
print 'Avg time for sirt algorithm is:', str(avg_sirt), ' in seconds'



Avg time for sirt algorithm is: 3.83759682178  in seconds


In [14]:
print 'Gridrec is : ' + str(avg_sirt/avg_gridrec) + 'faster than sirt algorithm'

Gridrec is : 1.4651804796faster than sirt algorithm


## Lightsource-(Sirt,Gridrec) Total time to completion vanilla 

In [6]:
def lightsource_reconstruction(message,algorithm='gridrec'):

    tt = time.time()

    msg_bin = binascii.unhexlify(message)
    tf = tempfile.NamedTemporaryFile(delete=True,suffix='.h5')
    tf.write(msg_bin)
    tf.flush()

    tt = time.time() - tt


    start_time = time.time()


    start = 0
    end = 2

    proj, flat, dark= dxchange.read_aps_32id(tf.name, sino=(start, end))
    theta = tomopy.angles(proj.shape[0])



    if (theta is None):
        theta = tomopy.angles(proj.shape[0])
    else:
        pass

    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm=algorithm)

    end_time = time.time()
    ttc = end_time - start_time

    #print 'Total time to completion of ' + algorithm  + '  algorithm is: ',  ttc
    #print 'Total time to decode is: ' + str(tt)

    tf.close()

    return 0



## create message:
start = time.time()
data_file = 'tooth.h5'
with open(data_file, "r") as f:
    data = f.read()
data_enc = binascii.hexlify(data).encode('utf-8')
end_data_generation = time.time()



avg_gridrec = 0
for i in range(10):
    aa = time.time()
    for i in range(1):
        lightsource_reconstruction(data_enc,'gridrec')
    avg_gridrec += time.time() - aa
    #print 'Sirt ttc is:', str(time.time() - aa)
avg_gridrec /= 10
print 'Avg time for gridrec algorithm is:', str(avg_gridrec), ' in seconds'




Avg time for gridrec algorithm is: 2.50613749027  in seconds


In [7]:
avg_sirt  = 0
for i in range(10):
    aa = time.time()
    for i in range(1):
        lightsource_reconstruction(data_enc,'sirt')

    avg_sirt += (time.time()-aa)
avg_sirt /= 10
print 'Avg time for sirt algorithm is:', str(avg_sirt), ' in seconds'


Avg time for sirt algorithm is: 3.96134998798  in seconds


In [11]:
print 'Gridrec is : ' + str(avg_sirt/avg_gridrec) + 'faster than sirt algorithm'

Gridrec is : 1.58065948231faster than sirt algorithm
